<a href="https://colab.research.google.com/github/sneha4948/Federated-Learning-for-Supply-Chain-Sensor-Data-Processing/blob/main/Hybrid_CNN%2BRNN_Classification_with_Simulated_data_flower_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing FLOWER, other requirements

In [1]:
!pip install -q "flwr[simulation]" flwr-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/19

In [2]:
# you might see a warning after running the command below, this can be ignored
# if you are running this outside Colab, you probably need to adjust the command below
# !pip install torch==1.13.1+cpu torchvision==0.14.1+cpu --extra-index-url https://download.pytorch.org/whl/cpu

In [3]:
!pip install matplotlib

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
# Flower Simulation code
import flwr as fl
from flwr.common import Context

In [6]:
# Random seed for reproducibility
np.random.seed(42)

# Parameters: Temperature, Humidity, CO2
n_samples = 1000  # Number of samples

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Dataset Simulation

In [7]:
# Generate random data within given ranges
temperature = np.random.uniform(8, 14, n_samples)
humidity = np.random.uniform(80, 95, n_samples)
co2 = np.random.uniform(600, 2000, n_samples)

# Function to label condition
def label_condition(temp, hum, co2):
    if temp > 12 and hum > 90 and co2 > 1500:
        return "Poor Condition"
    elif (temp > 10 and hum > 85) or (co2 > 1200):
        return "Average Condition"
    else:
        return "Good Condition"

# Create labels
labels = [label_condition(temp, hum, co2) for temp, hum, co2 in zip(temperature, humidity, co2)]

# Create a DataFrame
df = pd.DataFrame({
    'Temperature': temperature,
    'Humidity': humidity,
    'CO2': co2,
    'Condition': labels
})

# Save dataset to CSV
df.to_csv('container_conditions.csv', index=False)
print("Dataset created and saved as container_conditions.csv")


Dataset created and saved as container_conditions.csv


### Data Preprocessing for Classification

In [8]:
# Load dataset
data = pd.read_csv('container_conditions.csv')

# Encode the labels to integers
le = LabelEncoder()
data['Condition'] = le.fit_transform(data['Condition'])

# Split data into features and labels
X = data[['Temperature', 'Humidity', 'CO2']]
y = data['Condition']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### FedAvg, FedProx, FedBuff, FedAsync Strategy

In [9]:
from flwr.server.strategy import FedAvg, FedProx
from flwr.server.server import ServerConfig
from flwr.client import NumPyClient


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# Define FedBuff strategy
class FedBuffStrategy(FedAvg):
    def __init__(self, buffer_size=2, **kwargs):
        super().__init__(**kwargs)
        self.buffer_size = buffer_size
        self.buffer = []

    def aggregate_fit(self, rnd, results, failures):
        # Buffer the results
        self.buffer.extend(results)
        if len(self.buffer) >= self.buffer_size:
            # Aggregate when buffer is full
            aggregated_parameters, aggregated_metrics = super().aggregate_fit(rnd, self.buffer, failures)
            self.buffer = []  # Reset buffer
            return aggregated_parameters, aggregated_metrics
        else:
            # Return None to indicate that aggregation is not yet performed
            return None, {}


In [11]:
from flwr.server.strategy import Strategy
from flwr.common import Parameters, FitRes, EvaluateRes, Scalar
from typing import Optional, Tuple, List, Dict
import numpy as np

class FedAsyncStrategy(Strategy):
    def __init__(self, eta: float = 0.1):
        self.current_weights: Optional[List[np.ndarray]] = None
        self.eta = eta  # Learning rate for asynchronous updates

    def initialize_parameters(self, client_manager):
        # Initialize global model parameters
        if self.current_weights is None:
            # Create a new model to get initial weights
            model = create_model()
            self.current_weights = model.get_weights()
        parameters = fl.common.weights_to_parameters(self.current_weights)
        return parameters

    def configure_fit(
        self, rnd: int, parameters: Parameters, client_manager
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitIns]]:
        # Configure clients to train asynchronously
        config = {}
        # Sample all available clients
        clients = client_manager.all()
        fit_ins = fl.common.FitIns(parameters, config)
        instructions = [(client, fit_ins) for client in clients]
        return instructions

    def configure_evaluate(
        self, rnd: int, parameters: Parameters, client_manager
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.EvaluateIns]]:
        # Optionally, configure clients for evaluation
        # For this example, we will not perform evaluation
        return []

    def aggregate_fit(
        self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]], failures
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Update global weights asynchronously
        if not results:
            return None, {}

        for _, fit_res in results:
            client_weights = fl.common.parameters_to_weights(fit_res.parameters)
            if self.current_weights is None:
                self.current_weights = client_weights
            else:
                # Asynchronous update: weighted average
                self.current_weights = [
                    (1 - self.eta) * w_global + self.eta * w_client
                    for w_global, w_client in zip(self.current_weights, client_weights)
                ]

        parameters = fl.common.weights_to_parameters(self.current_weights)
        return parameters, {}

    def aggregate_evaluate(
        self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, EvaluateRes]], failures
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        # Optionally, aggregate evaluation results
        # Since we're not performing evaluation, return None
        return None, {}

    def evaluate(self, parameters: Parameters):
        # Optional: Implement server-side evaluation if needed
        return None


In [12]:
def start_federated_learning(strategy_name: str, num_rounds: int):
    """Start federated learning with the given number of rounds."""

    # Strategy for the server, using the FedAvg algorithm
    if strategy_name == 'fedavg':
        strategy = FedAvg(
            fraction_fit=1.0,  # Train on all clients each round
            fraction_evaluate=1.0,  # Evaluate on all clients each round
            min_fit_clients=5,  # Minimum number of clients to participate in training
            min_evaluate_clients=5,  # Minimum number of clients to participate in evaluation
            min_available_clients=5,  # Minimum number of available clients needed
        )

    elif strategy_name == 'fedprox':
        strategy = FedProx(
            fraction_fit=1.0,  # Train on all clients each round
            fraction_evaluate=1.0,  # Evaluate on all clients each round
            min_fit_clients=5,  # Minimum number of clients to participate in training
            min_evaluate_clients=5,  # Minimum number of clients to participate in evaluation
            min_available_clients=5,  # Minimum number of available clients needed
            proximal_mu=0.1,  # FedProx proximal term parameter (mu)
    )
    elif strategy_name == 'fedbuff':
        strategy = FedBuffStrategy(
                buffer_size=2,
                fraction_fit=1.0,
                fraction_evaluate=1.0,
                min_fit_clients=2,  # Adjust based on buffer_size
                min_evaluate_clients=2,
                min_available_clients=5,
            )
    elif strategy_name == 'fedasync':
        strategy = FedAsyncStrategy(eta=0.1)


    # Set the number of rounds in ServerConfig
    server_config = ServerConfig(num_rounds=num_rounds)

    # Run the federated learning simulation for `num_rounds`
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=5,  # Number of clients
        config=server_config,  # Pass the server configuration for the number of rounds
        client_resources={"num_cpus": 1}  # Adjust based on your system resources
    )


## Hybrid CNN+RNN classification

In [13]:
def create_cnn_rnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(3, 1)),
        tf.keras.layers.Conv1D(filters=32, kernel_size=2, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.LSTM(64, return_sequences=False),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

## Client

In [14]:
# Define Flower client
class ContainerClient(fl.client.NumPyClient):
    def __init__(self, model, X_train, y_train, X_test, y_test):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def get_parameters(self, config=None):  # Add config argument here
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.X_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.X_test, self.y_test)
        return accuracy, len(self.X_test), {"accuracy": accuracy, "loss": loss}


# Correct client_fn signature
def client_fn(context: Context):
    model = create_cnn_rnn_model()
    return NumPyClient.to_client(ContainerClient(model, X_train, y_train, X_test, y_test))

## FedAvg

In [15]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedavg', num_rounds=5)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:04:03,371	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources

1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 771ms/step - accuracy: 0.7188 - loss: 0.6503
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 0.7329  
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 0.7329  


(ClientAppActor pid=8336) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=8336)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=8335) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=8335)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 770ms/step - accuracy: 0.7188 - loss: 0.6503
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 0.7329  
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 0.6365   [repeated 3x across cluster]


(ClientAppActor pid=8336) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=8336)   warnings.warn( [repeated 3x across cluster]
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=8336) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=8336)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=8336) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` 

...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=8336) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=8336)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=8336) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=8336)   warnings.warn( [repeated 3x across cluster]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 0.6070 [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7188 - loss: 0.5026
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6768 - loss: 0.6070
1/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7188 - loss: 0.5026
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6768 - loss: 0.6070


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=8336) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=8336)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=8336) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=8336)   warnings.warn( [repeated 3x across cluster]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6768 - loss: 0.6070   [repeated 2x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.8125 - loss: 0.4699
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7024 - loss: 0.5579
1/7 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.8125 - loss: 0.4699
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7024 - loss: 0.5579
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7024 - loss: 0.5579


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 73.11s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.6899999976158142
INFO :      


## FedProx

In [16]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedprox', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:05:35,146	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__':

1/7 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.7188 - loss: 0.6484
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 0.7451
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 0.7451


(ClientAppActor pid=9374) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=9374)   warnings.warn( [repeated 3x across cluster]
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=9374) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=9374)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=9374) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` 

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6768 - loss: 0.7091 [repeated 4x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7188 - loss: 0.6144
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6768 - loss: 0.7091


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=9376) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=9376)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=9376) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=9376)   warnings.warn( [repeated 3x across cluster]


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=9374) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=9374)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=9374) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=9374)   warnings.warn( [repeated 3x across cluster]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6768 - loss: 0.6743   [repeated 3x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7188 - loss: 0.5742
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 0.6637


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=9376) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=9376)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=9374) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=9374)   warnings.warn( [repeated 3x across cluster]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 0.6322 [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.7188 - loss: 0.5431
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6768 - loss: 0.6322


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 76.45s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.675000011920929
INFO :      


## FedBuff

In [17]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedbuff', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:07:09,390	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 4005666816.0, 

1/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7188 - loss: 0.6573
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 0.7610
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 0.7610


(ClientAppActor pid=10411) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=10411)   warnings.warn( [repeated 3x across cluster]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 0.7610  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=10411) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=10411)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=10415) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=10415)   warnings.warn( [repeated 3x across cluster]


1/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7812 - loss: 0.4810
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7008 - loss: 0.5470   [repeated 3x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7008 - loss: 0.5557
1/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7812 - loss: 0.4810


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=10411) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=10411)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=10415) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=10415)   warnings.warn( [repeated 3x across cluster]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7008 - loss: 0.5557   [repeated 3x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7008 - loss: 0.5557
1/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7500 - loss: 0.4792
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7376 - loss: 0.5318
1/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7500 - loss: 0.4792


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 72.87s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.6899999976158142
INFO :      		round 4: 0.6899999976158142
INFO :      		round 5: 0.7250000238418579
INFO :      


## FedAsync

In [18]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedasync', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:08:39,516	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__inte

1/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7188 - loss: 0.6942
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 0.7677
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6768 - loss: 0.7677
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6768 - loss: 0.7677


(ClientAppActor pid=11448) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=11448)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=11448) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=11448)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 0.6487 [repeated 3x across cluster]


(ClientAppActor pid=11451) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 4x across cluster]
(ClientAppActor pid=11451)   warnings.warn( [repeated 4x across cluster]


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=11448) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=11448)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=11451) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=11451)   warnings.warn( [repeated 3x across cluster]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 0.6174   [repeated 3x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7188 - loss: 0.5453
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6768 - loss: 0.6317


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=11451) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=11451)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=11451) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=11451)   warnings.warn( [repeated 3x across cluster]


1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 831ms/step - accuracy: 0.7188 - loss: 0.5180
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 0.6174   [repeated 4x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 0.5973  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 71.24s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.675000011920929
INFO :      
